# ClearML integration

## MLOps: integrate clearml in your experiment
### it's very important to track all our experiemnt, results , data, code & model versioning for presentation.

## docs - How to use in colab
### "https://clear.ml/docs/latest/docs/guides/ide/google_colab/#:~:text=Go%20to%20your%20Settings%20page,the%20information%20that%20pops%20up.&text=For%20additional%20options%20for%20running,Orchestration%20page%20of%20your%20server."




### credential for clearml your experiment
     key='DIL5YU0YE1NNR1IDFPHIRGQVDTSUP8',
     secret='9lUrgFOsh6pz57mY54DgjekKCwF-7jCbBRSc8nAWnaNGPWkODPLK62qBQ89igm7O-yo'

### define your project name and task name or can use below.
#### create a sepate project name if you are doing independent experiment

In [ ]:
### define your project name and task name or can use below.
#### create a sepate project name if you are doing independent experiment

from clearml import Task

# Initialize ClearML task
task = Task.init(
    project_name="GenAI_OpenStack_RCA",  # Project name
    task_name="train_AnomalyModel",     # Task name
    task_type=Task.TaskTypes.training   # Task type
)

## How to run in colab

In [ ]:
!pip install git+https://github.com/allegroai/clearml
!pip install clearml-agent

In [3]:
! export MPLBACKEND=TkAg

In [4]:
from clearml import Task

Task.set_credentials(
     api_host="https://api.clear.ml",
     web_host="https://app.clear.ml",
     files_host="https://files.clear.ml",
     key='DIL5YU0YE1NNR1IDFPHIRGQVDTSUP8',
     secret='9lUrgFOsh6pz57mY54DgjekKCwF-7jCbBRSc8nAWnaNGPWkODPLK62qBQ89igm7O-yo'
)

In [ ]:
!clearml-agent daemon --queue default &

In [ ]:
# create example dataset
from clearml import StorageManager, Dataset

# # Download sample csv file
# csv_file = StorageManager.get_local_copy(
#     remote_url="https://vincentarelbundock.github.io/Rdatasets/csv/AER/Affairs.csv"
# )

# Create a dataset with ClearML`s Dataset class
dataset = Dataset.create(
    dataset_project="GenAI_OpenStack_RCA", dataset_name="OpenStack_2k.log_structured"
)

# add the example csv
dataset.add_files(path='/content/OpenStack_2k.log_structured.csv')

# Upload dataset to ClearML server (customizable)
dataset.upload()

# commit dataset changes
dataset.finalize()

In [ ]:
from clearml import Task

# Initialize ClearML task
task = Task.init(
    project_name="GenAI_OpenStack_RCA",  # Project name
    task_name="train_AnomalyModel",     # Task name
    task_type=Task.TaskTypes.training   # Task type
)

In [ ]:
#train a model with clearmltracking
from clearml import Logger
import clearml
from tensorflow.keras.callbacks import Callback

    def train_model(self, log_file, epochs=3, batch_size=16, learning_rate=2e-5):
        # Prepare data
        dataset = self.prepare_data(log_file, batch_size)

        # Compile the model
        optimizer = optimizers.Adam(learning_rate=learning_rate)
        self.model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

        # Create a custom ClearML callback
        class ClearMLCallback(Callback):
            def on_epoch_end(self, epoch, logs=None):
                # Log training metrics
                Logger.current_logger().report_scalar("loss", "train", iteration=epoch, value=logs["loss"])
                Logger.current_logger().report_scalar("accuracy", "train", iteration=epoch, value=logs["accuracy"])

        # Add ClearML callback
        clearml_callback = ClearMLCallback()

        # Train the model
        self.model.fit(dataset, epochs=epochs, callbacks=[clearml_callback])

        # Save and log the model artifact
        model_path = "trained_model.h5"
        self.model.save(model_path)  # Save the model locally
        task.upload_artifact("Trained_Model", artifact_object=model_path)  # Log model to ClearML

In [ ]:
# add simple test logs/metrics etc in clearml
from clearml import Task
from clearml import Logger
import clearml

def load_model():
    # Define the model architecture
    model = LogAnomalyDetector(base_model='bert-base-uncased')

    # Log the model structure (architecture) as text
    task.get_logger().report_text("Model Architecture:\n" + str(model))

    # Load the state dictionary into the model
    state_dict_path = './logbert_anomaly_model_tf/model.pth'
    model.load_state_dict(
      torch.load(state_dict_path, map_location=torch.device('cpu'), weights_only=True)
    )
    model.eval()

    # Log the model file as an artifact
    task.upload_artifact("LogAnomalyDetector Model State", artifact_object=state_dict_path)

    print("Model loaded and logged in ClearML:", model)
    return model




# train binary clasification model for anomaly - unsupervised model

In [17]:

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, optimizers
from clearml import Logger
import clearml
from tensorflow.keras.callbacks import Callback

class LogAnomalyDetector(tf.keras.Model):
    """
    Anomaly Detection Model based on BERT with TensorFlow/Keras
    """
    def __init__(self, base_model_name='bert-base-uncased', max_length=128, **kwargs): # Added **kwargs to handle extra arguments
        super(LogAnomalyDetector, self).__init__(**kwargs) # Pass extra arguments to the superclass
        self.max_length = max_length
        self.bert = TFBertModel.from_pretrained(base_model_name)
        self.anomaly_head = models.Sequential([
            layers.Dense(128, activation='relu'),
            layers.Dense(1, activation='sigmoid')
        ])

    def call(self, inputs, training=False):
        # Access input_ids and attention_mask using keys instead of unpacking
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        bert_outputs = self.bert(input_ids, attention_mask=attention_mask, training=training)
        cls_embedding = bert_outputs.last_hidden_state[:, 0, :]  # [CLS] token
        return self.anomaly_head(cls_embedding)


class LogAnomalyDetectionPipeline:
    """
    End-to-end pipeline for log anomaly detection using TensorFlow
    """
    def __init__(self, model_name='bert-base-uncased', max_length=128, save_path='./logbert_anomaly_model_tf'):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = LogAnomalyDetector(base_model_name=model_name, max_length=max_length)
        self.save_path = save_path
        self.max_length = max_length

    # def prepare_data(self, log_file, batch_size=16):
    #     # Load log data
    #     log_data = pd.read_csv(log_file)
    #     logs = log_data['EventTemplate'].tolist()

    #     # Tokenize logs
    #     inputs = self.tokenizer(
    #         logs,
    #         max_length=self.max_length,
    #         padding='max_length',
    #         truncation=True,
    #         return_tensors='tf'
    #     )

    #     dataset = tf.data.Dataset.from_tensor_slices((
    #         {'input_ids': inputs['input_ids'], 'attention_mask': inputs['attention_mask']},
    #         tf.zeros(len(logs))  # Dummy labels for unsupervised learning
    #     )).batch(batch_size)
    #     return

    def prepare_data(self, log_file, batch_size=16):
        # Load log data
        log_data = pd.read_csv(log_file)
        logs = log_data['EventTemplate'].tolist()

        # Tokenize logs
        inputs = self.tokenizer(
            logs,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='tf'
        )

        dataset = tf.data.Dataset.from_tensor_slices((
            {'input_ids': inputs['input_ids'], 'attention_mask': inputs['attention_mask']},
            tf.zeros(len(logs))  # Dummy labels for unsupervised learning
        )).batch(batch_size)
        return dataset # Added return statement to return the dataset


    def train_model(self, log_file, epochs=3, batch_size=16, learning_rate=2e-5):
        # Prepare data
        dataset = self.prepare_data(log_file, batch_size)

        # Compile the model
        optimizer = optimizers.Adam(learning_rate=learning_rate)
        self.model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

        # Create a custom ClearML callback
        class ClearMLCallback(Callback):
            def on_epoch_end(self, epoch, logs=None):
                # Log training metrics
                Logger.current_logger().report_scalar("loss", "train", iteration=epoch, value=logs["loss"])
                Logger.current_logger().report_scalar("accuracy", "train", iteration=epoch, value=logs["accuracy"])

        # Add ClearML callback
        clearml_callback = ClearMLCallback()

        # Train the model
        self.model.fit(dataset, epochs=epochs, callbacks=[clearml_callback])

        # Save and log the model artifact
        model_path = "trained_model.h5"
        self.model.save(model_path)  # Save the model locally
        task.upload_artifact("Trained_Model", artifact_object=model_path)  # Log model to ClearML


    # def train_model(self, log_file, epochs=3, batch_size=16, learning_rate=2e-5):
    #     # Prepare data
    #     dataset = self.prepare_data(log_file, batch_size)

    #     # Compile the model
    #     optimizer = optimizers.Adam(learning_rate=learning_rate)
    #     self.model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    #     # Train the model
    #     self.model.fit(dataset, epochs=epochs)

    def save_model(self):
        """Save the model and tokenizer"""
        os.makedirs(self.save_path, exist_ok=True)
        self.model.save(f"{self.save_path}/model.h5")
        self.tokenizer.save_pretrained(self.save_path)
        print(f"Model saved to {self.save_path}")

    def load_model(self):
        """Load the model and tokenizer"""
        self.model = tf.keras.models.load_model(f"{self.save_path}/model.h5", custom_objects={'LogAnomalyDetector': LogAnomalyDetector})
        self.tokenizer = BertTokenizer.from_pretrained(self.save_path)
        print("Model loaded successfully")

    def detect_anomaly(self, log_message, threshold=0.5):
        """
        Predict anomaly score for a single log message.
        """
        inputs = self.tokenizer(
            log_message,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='tf'
        )

        # Get prediction
        anomaly_score = self.model([inputs['input_ids'], inputs['attention_mask']])
        score = float(anomaly_score[0][0])

        return {
            'log_message': log_message,
            'anomaly_score': score,
            'is_anomaly': score > threshold
        }


In [18]:
from clearml import Task

# Initialize ClearML task
task = Task.init(
    project_name="GenAI_OpenStack_RCA",  # Project name
    task_name="train_AnomalyModel",     # Task name
    task_type=Task.TaskTypes.training   # Task type
)

def main():
    # Initialize pipeline
    print("Initialize pipeline")
    pipeline = LogAnomalyDetectionPipeline()

    # Train the model
    print("Train the model")
    pipeline.train_model(
        log_file='/content/OpenStack_2k.log_structured.csv',
        epochs=3
    )

    # # Save the model
    # print("Save the model")
    # pipeline.save_model()

    # # Load the model
    # print("Load the model")
    # pipeline.load_model()

    # Example anomaly detection
    print("Example anomaly detection")
    sample_log = "nova-compute.log.1.2017-05-16_13:55:31	2017-05-16	00:00:13.658	2931	INFO	nova.compute.resource_tracker	req-addc1839-2ed5-4778-b57e-5854eb7b8b09 - - - - -	Auditing locally available compute resources for node cp-1.slowvm1.tcloud-pg0.utah.cloudlab.us	E28	Auditing locally available compute resources for node <*>"
    anomaly_result = pipeline.detect_anomaly(sample_log)
    print(anomaly_result)

if __name__ == "__main__":
    main()

Initialize pipeline


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Train the model
Epoch 1/3
125/125 ━━━━━━━━━━━━━━━━━━━━ 56s 117ms/step - accuracy: 1.0000 - loss: 0.1782
Epoch 2/3
125/125 ━━━━━━━━━━━━━━━━━━━━ 19s 125ms/step - accuracy: 1.0000 - loss: 0.0269
Epoch 3/3
125/125 ━━━━━━━━━━━━━━━━━━━━ 15s 121ms/step - accuracy: 1.0000 - loss: 0.0105


Example anomaly detection


TypeError: Exception encountered when calling LogAnomalyDetector.call().

[1mlist indices must be integers or slices, not str[0m

Arguments received by LogAnomalyDetector.call():
  • inputs=['tf.Tensor(shape=(1, 128), dtype=int32)', 'tf.Tensor(shape=(1, 128), dtype=int32)']
  • training=False

# Previous experiment

In [2]:
import os
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
from typing import List, Dict, Any
from torch.utils.data import Dataset, DataLoader
from transformers import (
    BertModel,
    BertTokenizer,
    AdamW,
    get_linear_schedule_with_warmup
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

class LogDataset(Dataset):
    """Custom dataset for log entries"""
    def __init__(self, logs: List[str], tokenizer, max_length: int = 128):
        self.logs = logs
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.logs)

    def __getitem__(self, idx):
        log = self.logs[idx]
        encoding = self.tokenizer(
            log,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }

class LogAnomalyDetector(nn.Module):
    """Unsupervised Anomaly Detection Model"""
    def __init__(self, base_model: str = 'bert-base-uncased'):
        super().__init__()
        self.bert = BertModel.from_pretrained(base_model)
        self.anomaly_head = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, input_ids, attention_mask):
        # Extract BERT embeddings
        bert_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        # Use [CLS] token representation
        cls_embedding = bert_output.last_hidden_state[:, 0, :]

        # Anomaly scoring
        anomaly_score = self.anomaly_head(cls_embedding)
        return anomaly_score

class LogAnomalyTrainer:
    """Trainer for Unsupervised Anomaly Detection"""
    def __init__(
        self,
        model: nn.Module,
        learning_rate: float = 2e-5,
        device: str = None
    ):
        self.device = device or torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = model.to(self.device)
        self.optimizer = AdamW(self.model.parameters(), lr=learning_rate)

    def contrastive_loss(self, normal_embeddings, anomaly_embeddings):
        """Contrastive learning loss for anomaly detection"""
        normal_distances = torch.cdist(normal_embeddings, normal_embeddings)
        anomaly_distances = torch.cdist(anomaly_embeddings, anomaly_embeddings)

        # Maximize inter-cluster distance, minimize intra-cluster distance
        return torch.mean(normal_distances) - torch.mean(anomaly_distances)

    def train(
        self,
        dataloader: DataLoader,
        epochs: int = 15,
        verbose: bool = True
    ):
        for epoch in range(epochs):
            self.model.train()
            total_loss = 0

            for batch in dataloader:
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)

                # Compute anomaly scores
                anomaly_scores = self.model(input_ids, attention_mask)

                # Compute unsupervised loss
                loss = torch.mean(anomaly_scores)  # Encourage low anomaly scores

                # Backpropagation
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                total_loss += loss.item()

            if verbose:
                print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader)}")

class LogAnomalyDetectionPipeline:
    """End-to-end pipeline for log anomaly detection"""
    def __init__(
        self,
        model_name: str = 'bert-base-uncased',
        save_path: str = './logbert_anomaly_model'
    ):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = LogAnomalyDetector(base_model=model_name)
        self.save_path = save_path

    def prepare_data(
        self,
        log_file: str,
        batch_size: int = 4
    ) -> DataLoader:
        # Load log data
        log_data = pd.read_csv(log_file)
        logs = log_data['EventTemplate'].tolist()

        # Create dataset and dataloader
        dataset = LogDataset(logs, self.tokenizer)
        return DataLoader(dataset, batch_size=batch_size, shuffle=True)

    def train_model(
        self,
        log_file: str,
        epochs: int = 15,
        learning_rate: float = 2e-6
    ):
        # Prepare dataloader
        print("LogAnomalyDetectionPipeline: Prepare dataloader")
        dataloader = self.prepare_data(log_file)

        # Initialize trainer
        print("LogAnomalyDetectionPipeline: Initialize trainer")
        trainer = LogAnomalyTrainer(
            model=self.model,
            learning_rate=learning_rate
        )

        # Train the model
        print("LogAnomalyDetectionPipeline: Train the model")
        trainer.train(dataloader, epochs)

    def save_model(self):
        """Save the fine-tuned model"""
        os.makedirs(self.save_path, exist_ok=True)
        torch.save(self.model.state_dict(), f'{self.save_path}/model.pth')
        # torch.save(self.model, f'{self.save_path}/model2.pth')
        # torch.save(self.model, f'{self.save_path}/model2.pkl')
        # self.model.save(f"{self.save_path}/model.h5")
        self.tokenizer.save_pretrained(self.save_path)
        print(f"Model saved to {self.save_path}")

    # def load_model(self):
    #     """Load the saved model"""
    #     self.model.load_state_dict(torch.load(f'{self.save_path}/model.pth'))
    #     self.tokenizer = BertTokenizer.from_pretrained(self.save_path)
    #     self.model.eval()
    #     print("Model loaded successfully")

    def load_model(self):
        """Load the saved model"""
        # self.model.load_state_dict(torch.load(f'{self.save_path}/model.pth')) # Original line causing the error
        self.model = torch.load(f'{self.save_path}/model.pth') # Load the entire model
        self.tokenizer = BertTokenizer.from_pretrained(self.save_path)
        self.model.eval()
        print("Model loaded successfully")

    def detect_anomaly(self, log_message: str, threshold: float = 0.5) -> Dict[str, Any]:
        """
        Detect anomaly for a single log message

        Args:
            log_message (str): Log entry to check
            threshold (float): Anomaly score threshold

        Returns:
            Dict with anomaly detection results
        """
        # Tokenize input
        inputs = self.tokenizer(
            log_message,
            return_tensors='pt',
            padding=True,
            truncation=True
        ).to(self.model.bert.embeddings.word_embeddings.weight.device) # Move inputs to the same device as the model's word embeddings

        # Compute anomaly score
        with torch.no_grad():
            anomaly_score = self.model(
                inputs['input_ids'],
                inputs['attention_mask']
            )

        # Convert to numpy for easier processing
        score = anomaly_score.cpu().numpy()[0][0]

        return {
            'log_message': log_message,
            'anomaly_score': float(score),
            'is_anomaly': bool(score > threshold)
        }

# Example Usage
def main():
    # Initialize pipeline
    print("Initialize pipeline")
    pipeline = LogAnomalyDetectionPipeline()

    # Train the model
    print("Train the model")
    pipeline.train_model(
        log_file='/content/OpenStack_2k.log_structured.csv',
        epochs=3
    )

    # Save the model
    print("Save the model")
    pipeline.save_model()

    # Load the model
    print("Load the model")
    pipeline.load_model()

    # Example anomaly detection
    print("Example anomaly detection")
    sample_log = "nova-compute.log.1.2017-05-16_13:55:31	2017-05-16	00:00:13.658	2931	INFO	nova.compute.resource_tracker	req-addc1839-2ed5-4778-b57e-5854eb7b8b09 - - - - -	Auditing locally available compute resources for node cp-1.slowvm1.tcloud-pg0.utah.cloudlab.us	E28	Auditing locally available compute resources for node <*>"
    anomaly_result = pipeline.detect_anomaly(sample_log)
    print(anomaly_result)

if __name__ == "__main__":
    main()

Initialize pipeline
Train the model
LogAnomalyDetectionPipeline: Prepare dataloader
LogAnomalyDetectionPipeline: Initialize trainer


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


LogAnomalyDetectionPipeline: Train the model
Epoch 1/3, Loss: 0.066974714551121
Epoch 2/3, Loss: 0.009382738987915218
Epoch 3/3, Loss: 0.004826889187097549
Save the model
Model saved to ./logbert_anomaly_model
Load the model


<ipython-input-2-1c53249b26eb>:181: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model = torch.load(f'{self.save_path}/model.pth') # Load the entire model


AttributeError: 'collections.OrderedDict' object has no attribute 'eval'

In [ ]:
import random
from typing import List
import re

class LogGenerator:
    model_name: str = 'bert-base-uncased',
    save_path: str = './logbert_anomaly_model'
    @staticmethod
    def generate_standard_logs(num_logs: int = 20) -> List[str]:
        """
        Generate standard OpenStack Nova Compute log messages
        """
        standard_templates = [
            # Standard audit logs
            "nova-compute.log	{date}	{time}	{pid}	INFO	nova.compute.resource_tracker	{req_id}	Auditing locally available compute resources for node {node_name}",

            # Instance lifecycle logs
            "nova-compute.log	{date}	{time}	{pid}	INFO	nova.compute.manager	{req_id}	Creating instance {instance_id} on hypervisor",

            # Resource allocation logs
            "nova-compute.log	{date}	{time}	{pid}	INFO	nova.compute.resource_tracker	{req_id}	Allocated {resource_type} for instance {instance_id}",

            # Network configuration logs
            "nova-compute.log	{date}	{time}	{pid}	INFO	nova.network.manager	{req_id}	Configuring network for instance {instance_id}",
        ]

        logs = []
        for _ in range(num_logs):
            log = random.choice(standard_templates).format(
                date=f"2024-{random.randint(1, 12):02d}-{random.randint(1, 28):02d}",
                time=f"{random.randint(0, 23):02d}:{random.randint(0, 59):02d}:{random.randint(0, 59):02d}.{random.randint(0, 999):03d}",
                pid=random.randint(1000, 9999),
                req_id=f"req-{random.randint(10000, 99999)}",
                node_name=f"cp-{random.randint(1, 10)}.hypervisor.cluster",
                instance_id=f"instance-{random.randint(1000, 9999)}",
                resource_type=random.choice(['CPU', 'Memory', 'Storage', 'Network'])
            )
            logs.append(log)

        return logs

    @staticmethod
    def generate_anomalous_logs(num_logs: int = 5) -> List[str]:
        """
        Generate anomalous OpenStack Nova Compute log messages
        """
        anomalous_templates = [
            # Unusual error messages
            "nova-compute.log	{date}	{time}	{pid}	ERROR	nova.compute.manager	{req_id}	Unexpected error during instance creation: {error_details}",

            # Resource exhaustion
            "nova-compute.log	{date}	{time}	{pid}	CRITICAL	nova.compute.resource_tracker	{req_id}	Critical: No available resources for new instance allocation",

            # Unusual process terminations
            "nova-compute.log	{date}	{time}	{pid}	WARNING	nova.compute.manager	{req_id}	Compute service unexpectedly terminated",

            # Potential security-related anomalies
            "nova-compute.log	{date}	{time}	{pid}	ALERT	nova.security.monitor	{req_id}	Potential unauthorized access attempt detected",
        ]

        logs = []
        for _ in range(num_logs):
            log = random.choice(anomalous_templates).format(
                date=f"2024-{random.randint(1, 12):02d}-{random.randint(1, 28):02d}",
                time=f"{random.randint(0, 23):02d}:{random.randint(0, 59):02d}:{random.randint(0, 59):02d}.{random.randint(0, 999):03d}",
                pid=random.randint(1000, 9999),
                req_id=f"req-{random.randint(10000, 99999)}",
                error_details=random.choice([
                    "Memory allocation failure",
                    "Disk I/O error",
                    "Network configuration mismatch",
                    "Unexpected kernel panic"
                ])
            )
            logs.append(log)

        return logs



    @staticmethod
    def prepare_log_dataset(self, anomaly_detector):
        """
        Prepare a dataset of logs for testing
        """
        # Generate standard and anomalous logs
        standard_logs = LogGenerator.generate_standard_logs(50)
        anomalous_logs = LogGenerator.generate_anomalous_logs(20)

        # Combine logs
        all_logs = standard_logs + anomalous_logs
        random.shuffle(all_logs)

        # Detect anomalies
        anomaly_results = []
        for log in all_logs:
            result = anomaly_detector.detect_anomaly(log)
            anomaly_results.append(result)

        return anomaly_results

    @staticmethod
    def analyze_anomaly_detection(self, anomaly_results):
        """
        Analyze the anomaly detection results
        """
        total_logs = len(anomaly_results)
        detected_anomalies = [
            result for result in anomaly_results
            if result['is_anomaly']
        ]

        print("\n--- Anomaly Detection Analysis ---")
        print(f"Total Logs Analyzed: {total_logs}")
        print(f"Detected Anomalies: {len(detected_anomalies)}")
        print(f"Anomaly Detection Rate: {len(detected_anomalies)/total_logs*100:.2f}%")

        # Print top 5 most anomalous logs
        print("\nTop 5 Most Anomalous Logs:")
        top_anomalies = sorted(
            anomaly_results,
            key=lambda x: x['anomaly_score'],
            reverse=True
        )[:5]

        for anomaly in top_anomalies:
            print(f"Log: {anomaly['log_message']}")
            print(f"Anomaly Score: {anomaly['anomaly_score']}")
            print(f"Is Anomaly: {anomaly['is_anomaly']}\n")

    @staticmethod
    def load_model():
        """Load the saved model"""
        save_path: str = './logbert_anomaly_model'
        model.load_state_dict(torch.load(f'{save_path}/model.pth'))
        tokenizer = BertTokenizer.from_pretrained(save_path)
        model.eval()
        print("Model loaded successfully")

# Main Execution
def main():
    # Initialize Anomaly Detector (from previous implementation)
    anomaly_detector = LogGenerator()

    # Load pre-trained model (optional)
    # anomaly_detector.load_model()

    # # Train the model (uncomment if not pre-trained)
    # log_file = '/content/OpenStack_2k.log_structured.csv'
    # anomaly_detector.train(log_file, epochs=3)

    # # Save the model
    # anomaly_detector.save_model()
    anomaly_detector.load_model()

    # Specific log prediction
    sample_log = "nova-compute.log.1.2017-05-16_13:55:31	2017-05-16	00:00:13.658	2931	INFO	nova.compute.resource_tracker	req-addc1839-2ed5-4778-b57e-5854eb7b8b09 - - - - -	Auditing locally available compute resources for node cp-1.slowvm1.tcloud-pg0.utah.cloudlab.us	E28	Auditing locally available compute resources for node <*>"

    # Detect anomaly for specific log
    result = anomaly_detector.detect_anomaly(sample_log)
    print("\nSpecific Log Anomaly Detection:")
    print(result)

    # Comprehensive log analysis
    anomaly_results = prepare_log_dataset(self,anomaly_detector)
    analyze_anomaly_detection(anomaly_results)

if __name__ == "__main__":
    main()

NameError: name 'self' is not defined

In [ ]:
import pandas as pd
csv_file = "/content/OpenStack_2k.log_structured.csv"
log_data = pd.read_csv(csv_file)
# log_data

# New code start

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# Initialize the BERT tokenizer and model
model_name = "bert-base-uncased"  # Choose the BERT model you prefer
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Load the log data
log_data = pd.read_csv("/content/OpenStack_2k.log_structured.csv")

# Step 1: Batch Tokenization and Preprocessing
# Optimization: Use batch tokenization to speed up processing
logs = log_data["EventTemplate"].tolist()
tokenized = tokenizer(
    logs,
    padding=True,
    truncation=True,
    max_length=100,  # Set desired maximum length
    return_tensors="pt"
)

# Extract input IDs and attention masks
log_tensors = tokenized["input_ids"]
attention_masks = tokenized["attention_mask"]

# Step 2: Compute Log Embeddings
log_embeddings = []
batch_size = 32  # Process logs in batches to optimize GPU memory usage

for i in range(0, len(log_tensors), batch_size):
    input_ids_batch = log_tensors[i : i + batch_size]
    attn_mask_batch = attention_masks[i : i + batch_size]

    # Get embeddings for the batch
    with torch.no_grad():
        outputs = model(input_ids_batch, attention_mask=attn_mask_batch)
    # Extract the embedding for [CLS] token
    batch_embeddings = outputs.last_hidden_state[:, 0, :].numpy()
    log_embeddings.extend(batch_embeddings)# Step 3: Compute Anomaly Detection Using Cosine Similarity
# Check if log_embeddings has valid data
if len(log_embeddings) > 0:
    # Calculate the centroid (mean embedding) for all log embeddings
    mean_embedding = np.mean(log_embeddings, axis=0)
else:
    mean_embedding = None

# Define anomaly threshold (can be tuned)
anomaly_threshold = 0.95  # Adjust as needed for anomaly sensitivity

# Compute cosine similarity for each log embedding
anomalous_indices = []
if mean_embedding is not None:
    for i, log_embedding in enumerate(log_embeddings):
        similarity_score = cosine_similarity([log_embedding], [mean_embedding])[0][0]
        # Mark logs as anomalies if similarity score falls below the threshold
        if similarity_score < anomaly_threshold:
            anomalous_indices.append(i)
            print(f"Anomaly detected: Log {i} (Similarity: {similarity_score:.4f})")

# Add 'Anomaly' column to log_data
log_data["Anomaly"] = 0  # Initialize with all logs as normal
log_data.loc[anomalous_indices, "Anomaly"] = 1  # Mark anomalies in the DataFrame

# Save labeled data to a new file
output_file = "OpenStack_2k.log_labeled.csv"
log_data.to_csv(output_file, index=False)
print(f"Saved labeled data to {output_file}")



## new code end